The following tutorials are available from the [Wallaroo Tutorials Repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/notebooks_in_prod).

# Stage 4: Regular Batch Inference

In Stage 3: Deploy the Model in Wallaroo, the housing model created and tested in Stage 2: Training Process Automation Setup was uploaded to a Wallaroo instance and added to the pipeline `housing-pipe` in the workspace `housepricing`.  This pipeline can be deployed at any point and time and used with new inferences.

For the purposes of this demo, let's say that every month we find the newly entered and still-unsold houses and predict their sale price.

The predictions are entered into a staging table, for further inspection before being joined to the primary housing data table.

We show this as a notebook, but this can also be scripted and scheduled, using CRON or some other process.

## Resources

The following resources are used as part of this tutorial:

* **data**
  * `data/seattle_housing_col_description.txt`: Describes the columns used as part data analysis.
  * `data/seattle_housing.csv`: Sample data of the Seattle, Washington housing market between 2014 and 2015.
* **code**
  * `postprocess.py`: Formats the data after inference by the model is complete.
  * `simdb.py`: A simulated database to demonstrate sending and receiving queries.
  * `wallaroo_client.py`: Additional methods used with the Wallaroo instance to create workspaces, etc.
* **models**
  * `housing_model_xgb.onnx`: Model created in Stage 2: Training Process Automation Setup.

## Steps

This process will use the following steps:

* [Connect to Wallaroo](#connect-to-wallaroo): Connect to the Wallaroo instance and the `housepricing` workspace.
* [Deploy the Pipeline](#deploy-the-pipeline): Deploy the pipeline to prepare it to run inferences.
* [Read In New House Listings](#read-in-new-house-listings): Read in the previous month's house listings and submit them to the pipeline for inference.
* [Send Predictions to Results Staging Table](#send-predictions-to-results-staging-table): Add the inference results to the results staging table.

### Connect to Wallaroo

Connect to the Wallaroo instance and set the `housepricing` workspace as the current workspace.

In [1]:
import json
import pickle
import wallaroo
import pandas as pd
import numpy as np
import pyarrow as pa
import datetime

import simdb # module for the purpose of this demo to simulate pulling data from a database

from wallaroo_client import get_workspace

# used to display dataframe information without truncating
from IPython.display import display
pd.set_option('display.max_colwidth', None)


### Connect to the Wallaroo Instance

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [2]:
# Login through local Wallaroo instance

wl = wallaroo.Client()

In [3]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(name)
    return pipeline

In [4]:
workspace_name = 'housepricing'
model_name = "housepricemodel"
model_file = "./housing_model_xgb.onnx"
pipeline_name = "housing-pipe"

In [5]:
new_workspace = get_workspace(workspace_name)
_ = wl.set_current_workspace(new_workspace)

### Deploy the Pipeline

Deploy the `housing-pipe` workspace established in Stage 3: Deploy the Model in Wallaroo (`03_deploy_model.ipynb`).

In [6]:
pipeline = get_pipeline(pipeline_name)
pipeline.deploy()

name,housing-pipe
created,2023-09-05 18:08:50.854139+00:00
last_updated,2023-09-05 19:05:23.216037+00:00
deployed,True
tags,
versions,"671acb30-b6f3-4b24-83ed-390b9aaa29fd, 7341d56f-d9e0-45e6-ad07-a36b136264d1, d7f48b2c-93f4-48d3-80b6-399c30eeb3d7, 8f199fb9-cb4b-4e2b-9a91-35c002016018, dc3351f3-408d-46a4-b761-d22d24f83a2b, 6006adf0-3244-4490-9fd6-229a86c72a73, c81ee0b1-bb26-40ea-bba0-c1eab788f41f, fa1f1348-0b8a-46c7-8c4c-dde666a2ca41"
steps,housepricemodel
published,False


### Read In New House Listings

From the data store, load the previous month's house listing, prepare it as a DataFrame, then submit it for inferencing.

In [7]:
conn = simdb.simulate_db_connection()

# create the query
query = f"select * from {simdb.tablename} where date > DATE(DATE(), '-1 month') AND sale_price is NULL"
print(query)

# read in the data
newbatch = pd.read_sql_query(query, conn)
newbatch.shape

select * from house_listings where date > DATE(DATE(), '-1 month') AND sale_price is NULL


(1090, 22)

In [8]:
_vars = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view',
'condition', 'grade', 'sqft_above', 'sqft_basement', 'lat', 'long', 'sqft_living15', 'sqft_lot15', 'house_age', 'renovated', 'yrs_since_reno']

def create_features(housing_data):
    thisyear = datetime.datetime.now().year
    housing_data['house_age'] = thisyear - housing_data['yr_built']
    housing_data['renovated'] =  np.where((housing_data['yr_renovated'] > 0), 1, 0) 
    housing_data['yrs_since_reno'] =  np.where(housing_data['renovated'], housing_data['yr_renovated'] - housing_data['yr_built'], 0)
    return pd.DataFrame({
        "tensor": housing_data.loc[:, _vars].to_numpy().tolist()
        }
    )

evaluation_frame = create_features(newbatch)
evaluation_frame

,tensor
0,"[3.0, 1.75, 1250.0, 5963.0, 1.0, 0.0, 0.0, 4.0, 7.0, 1250.0, 0.0, 47.6796, -122.301, 970.0, 5100.0, 70.0, 0.0, 0.0]"
1,"[4.0, 1.0, 1610.0, 2982.0, 1.5, 0.0, 0.0, 4.0, 7.0, 1610.0, 0.0, 47.587, -122.294, 1610.0, 4040.0, 98.0, 0.0, 0.0]"
2,"[4.0, 2.25, 2010.0, 9603.0, 1.0, 0.0, 0.0, 3.0, 8.0, 1440.0, 570.0, 47.5343, -122.054, 2060.0, 9793.0, 37.0, 0.0, 0.0]"
3,"[3.0, 1.0, 1610.0, 8579.0, 1.0, 0.0, 0.0, 4.0, 7.0, 1010.0, 600.0, 47.4563, -122.171, 1610.0, 8579.0, 61.0, 0.0, 0.0]"
4,"[3.0, 1.0, 1300.0, 8316.0, 1.0, 0.0, 0.0, 4.0, 6.0, 1300.0, 0.0, 47.3221, -122.216, 1260.0, 8316.0, 69.0, 0.0, 0.0]"
...,...
1085,"[4.0, 2.75, 2460.0, 8643.0, 2.0, 0.0, 0.0, 3.0, 9.0, 2460.0, 0.0, 47.4828, -122.133, 3110.0, 8626.0, 12.0, 0.0, 0.0]"
1086,"[4.0, 2.5, 3650.0, 7090.0, 2.0, 0.0, 0.0, 3.0, 10.0, 3650.0, 0.0, 47.606, -122.052, 3860.0, 7272.0, 15.0, 0.0, 0.0]"
1087,"[3.0, 2.25, 1610.0, 3764.0, 2.0, 0.0, 0.0, 3.0, 7.0, 1610.0, 0.0, 47.3589, -122.083, 1610.0, 3825.0, 11.0, 0.0, 0.0]"
1088,"[4.0, 3.5, 2850.0, 5577.0, 2.0, 0.0, 0.0, 3.0, 8.0, 1950.0, 900.0, 47.5252, -122.192, 2850.0, 5708.0, 9.0, 0.0, 0.0]"


In [9]:
# query = {'query': newbatch.to_json()}

result = pipeline.infer(evaluation_frame)
# display(result)
predicted_prices = pd.DataFrame(result['out.variable'].apply(lambda x: x[0])).rename(columns={'out.variable':'prediction'})
display(predicted_prices[0:5])

,prediction
0,508255.0
1,500198.0
2,539598.0
3,270739.0
4,191304.0


### Send Predictions to Results Staging Table

Take the predicted prices based on the inference results so they can be joined into the `house_listings` table.

Once complete, undeploy the pipeline to return the resources back to the Kubernetes environment.

In [10]:
result_table = pd.DataFrame({
    'id': newbatch['id'],
    'saleprice_estimate': predicted_prices['prediction']
})

display(result_table)

result_table.to_sql('results_table', conn, index=False, if_exists='append')

,id,saleprice_estimate
0,9215400105,508255.0
1,1695900060,500198.0
2,9545240070,539598.0
3,1432900240,270739.0
4,6131600075,191304.0
...,...,...
1085,3304300300,577492.0
1086,6453550090,882930.0
1087,1760650820,271484.0
1088,3345700207,537434.0


In [11]:
# Display the top of the table for confirmation
pd.read_sql_query("select * from results_table limit 5", conn)

,id,saleprice_estimate
0,9215400105,508255.0
1,1695900060,500198.0
2,9545240070,539598.0
3,1432900240,270739.0
4,6131600075,191304.0


In [12]:
conn.close()
pipeline.undeploy()

name,housing-pipe
created,2023-09-05 18:08:50.854139+00:00
last_updated,2023-09-05 19:05:23.216037+00:00
deployed,False
tags,
versions,"671acb30-b6f3-4b24-83ed-390b9aaa29fd, 7341d56f-d9e0-45e6-ad07-a36b136264d1, d7f48b2c-93f4-48d3-80b6-399c30eeb3d7, 8f199fb9-cb4b-4e2b-9a91-35c002016018, dc3351f3-408d-46a4-b761-d22d24f83a2b, 6006adf0-3244-4490-9fd6-229a86c72a73, c81ee0b1-bb26-40ea-bba0-c1eab788f41f, fa1f1348-0b8a-46c7-8c4c-dde666a2ca41"
steps,housepricemodel
published,False


From here, organizations can automate this process.  Other features could be used such as data analysis using Wallaroo assays, or other features such as shadow deployments to test champion and challenger models to find which models provide the best results.